In [16]:
import pandas as pd
import numpy as np
import src.in_out as io
import src.info as info
import src.auxiliary_functions as aux
import src.condition as con
from src.spread import group_by_spread

## Veri Hazırlama

In [2]:
#Veriler, tarih ve ürün adları ile okunuyor ve birleştiriliyor
path_list=io.get_path('data')
cols= ["Time","BID price","ASK price"]
master_data=pd.concat((pd.read_excel(file,usecols=cols).\
                       assign(name=info.get_productName(file),date=info.get_productDate(file)
                             ) for file in path_list)).reset_index(drop=True)
master_data.columns=['time','bid_price','ask_price','name','date']

In [3]:
#date sütunu Timestamp'e dönüştürülüyor
master_data['date']=pd.to_datetime(master_data['date'], errors='coerce')

In [4]:
#tarih ve saat bilgisi birleştiriliyor
master_data['date'] += pd.to_timedelta(master_data.pop('time').astype(str))

In [5]:
#zaman dilimi ekleniyor
master_data['time_period']=master_data.date.dt.hour

## Mid Price

In [6]:
# mid price hesaplanıyor
master_data['mid_price']=aux.get_mid_price(master_data)

In [7]:
# spread hesaplama için master_data, uygun bir forma dönüştürülüyor
df_mid_price=master_data.pivot(index='date', columns='name', values='mid_price')

## Spread
<img src="img/spread_flowchart.png">

In [8]:
values={'a_PNLTICK':1,
'a_TICKSIZE':1,
'b_PNLTICK':1,
'b_TICKSIZE':1}

In [43]:
# pair'lar alınıyor
pairs=combinations(df_mid_price.columns, 2)

In [17]:
#spread hesaplanıyor
spread_list=[
    group_by_spread(df_mid_price.loc[:,pair],values) \
        for pair in pairs]
df_spread=pd.concat(spread_list,axis=1)

In [38]:
g=group_by_date_and_time(df_mid_price)

In [39]:
from src.spread import get_spread

In [44]:
slist=[
    group_by_date_and_time(df_mid_price.loc[:,pair]).apply(lambda x: get_spread(x,values))\
        for pair in pairs]


In [45]:
sprd=pd.concat(slist,axis=1)

In [48]:
sprd=sprd.droplevel([0,1])

In [ ]:
spread_series=mp_group.apply(lambda x : get_spread(x,values))
    df_spread=pd.DataFrame(spread_series).droplevel([0,1])
    return df_spread

In [22]:
x=df_spread.reset_index()

In [26]:
x['hour']=x.date.dt.hour

In [31]:
x[x['hour']==0]

,date,6AU8_6BU8,6AU8_6CU8,6BU8_6CU8,hour
0,2018-09-06 00:00:00,0.000000,0.0,0.000000,0
1,2018-09-06 00:00:01,0.000000,0.0,0.000000,0
2,2018-09-06 00:00:02,0.000000,0.0,0.000000,0
3,2018-09-06 00:00:03,0.000000,0.0,0.000000,0
4,2018-09-06 00:00:04,-0.000109,0.0,0.000109,0
...,...,...,...,...,...
43195,2018-09-07 00:59:55,-0.000300,0.0,0.000300,0
43196,2018-09-07 00:59:56,-0.000300,0.0,0.000300,0
43197,2018-09-07 00:59:57,-0.000300,0.0,0.000300,0
43198,2018-09-07 00:59:58,-0.000300,0.0,0.000300,0


In [28]:
x.groupby('hour').apply(aux.find_change)

date  6AU8_6BU8  6AU8_6CU8  6BU8_6CU8  hour
hour                                                      
0    1     00:00:01   0.000000        0.0   0.000000   0.0
     2     00:00:01   0.000000        0.0   0.000000   0.0
     3     00:00:01   0.000000        0.0   0.000000   0.0
     4     00:00:01  -0.000109        0.0   0.000109   0.0
     5     00:00:01   0.000000        0.0   0.000000   0.0
...             ...        ...        ...        ...   ...
23   79195 00:00:01   0.000000        0.0   0.000000   0.0
     79196 00:00:01   0.000000        0.0   0.000000   0.0
     79197 00:00:01   0.000000        0.0   0.000000   0.0
     79198 00:00:01   0.000000        0.0   0.000000   0.0
     79199 00:00:01   0.000000        0.0   0.000000   0.0

[79189 rows x 5 columns]

In [167]:
class SpreadGgraph:
    